In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
from nltk.corpus import stopwords

In [4]:
lyric_df = pd.read_csv('trainclean.csv',  usecols=range(0,5), header = 0, delimiter=",", quoting=csv.QUOTE_NONE, 
                       encoding='utf-8')
lyric_df.columns = ['artist', 'song', 'genre', 'lang', 'lyrics']

In [5]:
# for now, filtering out just two genres before the expensive stuff below
lyrics_metal = lyric_df.loc[(lyric_df['genre'] == 'Metal') & (lyric_df['lang'] == 'en')]
lyrics_metal = lyrics_metal.iloc[100: 2600] # there were much more metal songs than r&b, sliced to make it more even 
lyrics_rb = lyric_df.loc[(lyric_df['genre'] == 'R&B') & (lyric_df['lang'] == 'en')]

lyric_df = pd.concat([lyrics_metal, lyrics_rb])

genres = [i for i in lyric_df['genre'].unique() if isinstance(i, str)] # there were a few nan values to remove
# there are also a few rogue non genre values in the column that would need to be handled if using all genres or randomly getting a genre etc
# the actual genres from the above list are: 'Rock', 'Metal', 'Pop', 'Indie', 'R&B', 'Folk', 'Electronic', 'Jazz'

# below filtering to have only lyrics classed as english
all_lyrics = lyric_df.loc[lyric_df['lang'] == 'en']['lyrics'].astype(str).to_list()

# some cleaning- removing multiple back to back spaces
all_words = " ".join(" ".join(all_lyrics).split())
# removing special characters and lowercasing
all_words = re.sub(r'[^\w\s]', '', all_words).lower()

In [6]:
genres_words_dict = {}
for i in genres:
  genre_words = " ".join(lyric_df.loc[(lyric_df['genre'] == i) & (lyric_df['lang'] == 'en')]['lyrics'].astype(str).to_list())
  genre_words = " ".join(genre_words.split())

  genre_words = re.sub(r'[^\w\s]', '', genre_words).lower()
  genres_words_dict[i] = genre_words

In [7]:
# below is code used to make the tf and idf data and store into files, after files made just imported data from there

In [8]:
# import math
# # setting idfs based on how many different songs a word appears in out of the total

# idfs = {}
# for i in list(set(all_words.split(" "))):
#   count = 0 
#   for j in all_lyrics:
#     if i in j.split(" "):
#       count += 1
#     divided = count/len(all_lyrics)
#   try:
#     idfs[i] = math.log(divided) if divided != 0 else -10000
#   except:
#     print("problem at: " + str(i))

In [9]:
# # setting of idfs is very expensive, so saving to file to not have to repeat
# idfs_str = str(idfs)
# with open('Idfs.py', 'w') as file:
#     file.write("idfs = " + idfs_str)

In [10]:
# word_freq_dict_genre = {}
# for i in ['Metal', 'R&B']:
#   genre_dict = {}
#   wordset = list(set(genres_words_dict.get(i).split(" ")))
#   word_list = genres_words_dict.get(i).split(" ")
#   total_words = len(word_list)
#   for j in wordset:
#     count = word_list.count(j)
#     genre_dict[j] = count/total_words
#   word_freq_dict_genre[i] = genre_dict

In [11]:
# the above is slow but useable for two genres, saving to a file so it doesn't need to be run twice
# word_freq_dict_genre_str = str(word_freq_dict_genre)
# with open('Genre_Tfs.py', 'w') as file:
#     file.write("genre_tfs = " + word_freq_dict_genre_str)

In [4]:
# importing from py files storing variables
from Genre_Tfs import genre_tfs 
from Idfs import idfs

In [5]:
import math
tf_idfs_metal = {}
sorted_tfs_metal = sorted(genre_tfs.get('Metal').items(), key=lambda x: x[1], reverse=True)
for i in genre_tfs.get('Metal'):
  tf_idf = genre_tfs.get('Metal').get(i) * (idfs.get(i) if idfs.get(i)!=-10000 else 0)
  tf_idfs_metal[i] = tf_idf
sorted_metal = sorted(tf_idfs_metal.items(), key=lambda x: x[1])

In [8]:
metal_no_stopwords = {k: v for k, v in tf_idfs_metal.items() if k not in stopwords.words('english') and len(k)>0}

In [15]:
metal_no_stopwords_str = str(sorted(metal_no_stopwords.items(), key=lambda x: x[1])[:100])
with open('Metal_Tf_idfs.py', 'w') as file:
    file.write(metal_no_stopwords_str)

In [9]:
import math
tf_idfs_rb = {}
sorted_tfs_rb = sorted(genre_tfs.get('R&B').items(), key=lambda x: x[1], reverse=True)
for i in genre_tfs.get('R&B'):
  tf_idf = genre_tfs.get('R&B').get(i) * (idfs.get(i) if idfs.get(i)!=-10000 else 0)
  tf_idfs_rb[i] = tf_idf
sorted_rb = sorted(tf_idfs_rb.items(), key=lambda x: x[1], reverse=True)

In [10]:
rnb_no_stopwords = {k: v for k, v in tf_idfs_rb.items() if k not in stopwords.words('english') and len(k)>0}

In [18]:
rnb_no_stopwords_str = str(sorted(rnb_no_stopwords.items(), key=lambda x: x[1]))
with open('R&B_Tf_idfs.py', 'w') as file:
    file.write(rnb_no_stopwords_str)

In [11]:
# top words from each genre, manually removed few words that would class as should have been stopwords or are repeated in both
metal_top = [ ('well', -0.005044585661359806), ('oh', -0.004794831092790529), ('away', -0.004737480647955148), ('never', -0.004665590314842689), ('let', -0.004565950198802495), ('go', -0.004452147361580444), ('take', -0.004143752078275229), ('one', -0.004051101674227644), ('time', -0.004004688157161767), ('life', -0.003939865531221631), ('end', -0.0036500014948693113), ('know', -0.0035908448915495505), ('find', -0.003559023568782813), ('see', -0.0035133535088879516), ('back', -0.003499770675231604), ('us', -0.003490381125320545), ('feel', -0.003458040839772124), ('come', -0.0034483603300894555), ('inside', -0.003441367532121765), ('die', -0.003380690129252322), ('way', -0.003359028846347693), ('eyes', -0.003351004847405425), ('nothing', -0.003348030884207026), ('world', -0.0033192240292194307), ('wanna', -0.0033154013496469415), ('alone', -0.0032735074481889163), ('left', -0.0032553396039164384), ('fight', -0.0032496887565243455), ('lost', -0.0032212033054270653), ('god', -0.0032170309077624195), ('still', -0.0031408586149979894), ('lets', -0.0030341722586742), ('live', -0.0029926926208485596), ('want', -0.0029778647332209616), ('every', -0.0029746378772916872), ('say', -0.0029717946230236494), ('another', -0.002957139374261443), ('dead', -0.002927797386807331), ('fall', -0.00288147680964528), ('tonight', -0.002837296637617445), ('break', -0.0028231828380851126), ('make', -0.0028215353045461748), ('last', -0.002807371040772317), ('give', -0.0028027329750173443), ('run', -0.0028015178214603853), ('could', -0.0027703435353274826), ('hell', -0.0027564419867103502), ('tell', -0.0027520896143912754), ('mind', -0.002739115927111357), ('believe', -0.0027350466908928374), ('keep', -0.002725617126539209), ('home', -0.002717765806636134), ('leave', -0.002678020803211453), ('alive', -0.0026753073216455943), ('save', -0.0026666451519456536), ('heart', -0.002658988738187305), ('hold', -0.0026542558345093777), ('forever', -0.0026068726381557), ('always', -0.0025944856385915704), ('need', -0.002577206859604833), ('light', -0.002557544045600634), ('love', -0.00254222890076918), ('try', -0.002516502882956565), ('everything', -0.0025119404610205935), ('look', -0.0025049836711992035), ('think', -0.0025012606499788988), ('fire', -0.0024856472847253545), ('gone', -0.0024652901381730682), ('got', -0.002407505051887954), ('face', -0.002384767184913147), ('hate', -0.002368479235555012), ('pain', -0.002368051310220422), ('words', -0.0023622178224556815), ('right', -0.0023590751321486843), ('broken', -0.0023414242991772953), ('without', -0.002314628701425852), ('stay', -0.0022705696196186377), ('day', -0.002259034154793912)]
rb_top = [ ('faded', -0.026689231564648135), ('aint', -0.024974006230552664), ('big', -0.024685417691300785), ('london', -0.024534857457969253), ('blue', -0.01673581750040571), ('reggae', -0.01570246733474084), ('selfish', -0.013090087781192293), ('stripping', -0.012989042183630782), ('pussy', -0.012439133712564917), ('friday', -0.011845863765994809), ('swear', -0.0111055766935284), ('everybody', -0.010394020565657035), ('warning', -0.009599397706207682), ('come', -0.009405909063291149), ('falling', -0.009390831327238334), ('got', -0.009360287990171896), ('know', -0.009077092705855315), ('way', -0.00896354777803252), ('call', -0.008739921325382418), ('phone', -0.008447385609358406), ('bitch', -0.008293540694032483), ('minute', -0.008270517017600955), ('roam', -0.007902924822019616), ('texting', -0.00785123366737042), ('two', -0.007707609353387458), ('talking', -0.007588962649474621), ('late', -0.0071298396881955065), ('shit', -0.006838840683880651), ('wondering', -0.0067140954781070945), ('nights', -0.006686559462598407), ('mean', -0.0066319446721197555), ('far', -0.006629624626120177), ('even', -0.006454772552139463), ('memories', -0.006425217869783964), ('historical', -0.006280986933896336), ('used', -0.006249562786104186), ('real', -0.005945628472801855), ('kids', -0.005810762306879215), ('horny', -0.0057729076371692365), ('play', -0.0057618661314138295), ('figures', -0.005741480054584964), ('outside', -0.005725339681811998), ('feel', -0.005689630656006481), ('gon', -0.005654151687529509), ('ya', -0.005599061166214209), ('hysterical', -0.005475700301191106), ('going', -0.005308363563610024), ('stressing', -0.005264828340442137), ('drinking', -0.005153378637897779), ('popping', -0.0051012633403950715), ('need', -0.005062721742318241), ('fuckin', -0.0050616767334108964), ('fall', -0.00501286889708915), ('baby', -0.004876182952557411), ('text', -0.004854628019433947), ('party', -0.004766058185784295), ('amnesia', -0.004710740200422252), ('hit', -0.004699456023988918), ('sing', -0.004485172201488141), ('arms', -0.004479248932971367), ('inside', -0.004415409473526504),  ('fuck', -0.004182514401716713), ('run', -0.004120008845352809), ('yall', -0.004106775225893329), ('ass', -0.004069728824924853), ('help', -0.003999552526581899), ('tell', -0.003995553758054919), ('running', -0.003988410550296394), ('good', -0.0039717063139983804), ('disaster', -0.003921539746893806), ('long', -0.0038725725785325655), ('home', -0.0038503531216120984), ('taking', -0.003841300285139361), ('night', -0.0038289327983842196), ('name', -0.0037458255723295205), ('time', -0.0036922975295750676), ('ball', -0.003654255075031678), ('sex', -0.003654255075031678)]

In [12]:
# loading test data to make predictions on
lyric_df_test = pd.read_csv('testclean.csv',  usecols=range(0,5), header = 0, delimiter=",", quoting=csv.QUOTE_NONE, 
                       encoding='utf-8')
lyric_df_test.columns = ['song', 'year', 'artist', 'genre', 'lyrics']

In [50]:
# filter to just the two genres classifying between
lyrics_metal_test = lyric_df_test.loc[lyric_df_test['genre'] == 'Metal']
lyrics_rb_test = lyric_df_test.loc[lyric_df_test['genre'] == 'R&B']

lyric_df_test = pd.concat([lyrics_metal_test, lyrics_rb_test]).sample(frac = 0.5)

In [45]:
def predict_all(data, tfidfs):
  predictions_list = []
  for i, row in data.iterrows():
    lyrics = row['lyrics']
    # apply same basic preprocessing done to the training data
    words = " ".join(lyrics.split())
    words = re.sub(r'[^\w\s]', '', words).lower()
    # words for each song is just a single string at this point
    predictions_list.append(predict(words, tfidfs))
  return predictions_list
  

In [46]:
# tfidfs into one thing that can then differentiate by class
tfidfs = [{'Metal': metal_top}, {'R&B': rb_top}]

In [33]:
def predict(lyrics, tfidfs):
  word_list = lyrics.split(" ")
  class1 = list(tfidfs[0].keys())[0]
  class2 = list(tfidfs[1].keys())[0]
  class1_score = 0
  class2_score = 0
  class1_keywords = tfidfs[0].get(class1)
  class2_keywords = tfidfs[1].get(class2)
  for i in word_list:
    for word, score in class1_keywords:
      if i == word:
        class1_score += score
  for i in word_list:
    for word, score in class2_keywords:
      if i == word:
        class2_score += score
  if class1_score < class2_score:
    return class1
  else:
    return class2

In [34]:
def get_accuracy(data, predictions):
  real = data['genre'].tolist()
  total = len(real)
  correct = 0
  for i in range(total):
    if real[i] == predictions[i]:
      correct += 1
  return correct / total 

In [51]:
predictions = predict_all(lyric_df_test, tfidfs)

In [52]:
accuracy = get_accuracy(lyric_df_test, predictions)

In [53]:
print(str(round(accuracy *100, 2)) + "%")

65.0%
